In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import shutil
from yolo_tools import mseg_class_update, mseg2seg, random_select, ref_split, data_merge, seg_filter_and_remove, data_check

from yolo_mask_crop import myolo_crop
from classify_tools import ref_split_class
from yolo_sta import yolo_sta
from select_defect_data import obj_record, select_by_predict
from yolo_result_analysis import model_pred_compare


In [ ]:
defect_list = ['deformation', 'broken', 'abandonment', 'corrosion']
level_list = ['no', 'medium', 'high']

ps_mseg_c6_dir = r'/localnvme/data/billboard/ps_data/psdata735_mseg_c6_check0618'
ps_mseg_c6_update_dir = r'/localnvme/data/billboard/ps_data/psdata735_mseg_c6_check0618_update'


bd_mseg_c6_dir = r'/localnvme/data/billboard/bd_data/data626_mseg_c6_check0618'
bd_mseg_c6_update_dir = r'/localnvme/data/billboard/bd_data/data626_mseg_c6_check0618_update'

fuse_mseg_c6_dir = r'/localnvme/data/billboard/fused_data/data1361_mseg_c6_check0618'
fuse_mseg_c6_update_dir = r'/localnvme/data/billboard/fused_data/data1361_mseg_c6_check0618_update'

In [ ]:
seg_filter_and_remove(
    input_dir=bd_mseg_c6_dir,
    output_dir=bd_mseg_c6_update_dir,
    remove_class=6,
    cp_img=True,
    threshold=0.01,
    class_list=[1,3,5],
    with_attribute=True
)

In [ ]:
ref_split(
    ref_path = os.path.join(bd_mseg_c6_dir, 'val.txt'),
    img_dir = os.path.join(bd_mseg_c6_update_dir, 'images'),
    label_dir = os.path.join(bd_mseg_c6_update_dir, 'labels'),
    add_suffix='',
)

In [ ]:
seg_filter_and_remove(
    input_dir=ps_mseg_c6_dir,
    output_dir=ps_mseg_c6_update_dir,
    remove_class=6,
    cp_img=True,
    threshold=0.01,
    class_list=[1,3,5],
    with_attribute=True
)

In [ ]:
ref_split(
    ref_path = os.path.join(ps_mseg_c6_dir, 'val.txt'),
    img_dir = os.path.join(ps_mseg_c6_update_dir, 'images'),
    label_dir = os.path.join(ps_mseg_c6_update_dir, 'labels'),
    add_suffix='',
)

In [ ]:
attribute_path = os.path.join(bd_mseg_c6_dir, 'attribute.yaml')
check_result_ps1 = data_check(ps_mseg_c6_dir, mdet=True, attribute_path=attribute_path)
check_result_bd1 = data_check(bd_mseg_c6_dir, mdet=True, attribute_path=attribute_path)
check_result_ps2 = data_check(ps_mseg_c6_dir, mdet=True, attribute_path=attribute_path, check_item='attribute')
check_result_bd2 = data_check(bd_mseg_c6_dir, mdet=True, attribute_path=attribute_path, check_item='attribute')

if len(check_result_ps1+check_result_bd1 + check_result_ps2+check_result_bd2) == 0:
    data_merge(ps_mseg_c6_dir,
               bd_mseg_c6_dir,
               fuse_mseg_c6_dir)
    shutil.copy2(
        os.path.join(bd_mseg_c6_dir, 'class.txt'),
        os.path.join(fuse_mseg_c6_dir, 'class.txt'),
    )
    shutil.copy2(
        os.path.join(bd_mseg_c6_dir, 'attribute.yaml'),
        os.path.join(fuse_mseg_c6_dir, 'attribute.yaml'),
    )
else:
    print(f'find {check_result_ps1} and {check_result_bd1}')
    print(f'find {check_result_ps2} and {check_result_bd2}')

In [ ]:
attribute_path = os.path.join(bd_mseg_c6_dir, 'attribute.yaml')
check_result_ps1 = data_check(ps_mseg_c6_update_dir, mdet=True, attribute_path=attribute_path)
check_result_bd1 = data_check(bd_mseg_c6_update_dir, mdet=True, attribute_path=attribute_path)
check_result_ps2 = data_check(ps_mseg_c6_update_dir, mdet=True, attribute_path=attribute_path, check_item='attribute')
check_result_bd2 = data_check(bd_mseg_c6_update_dir, mdet=True, attribute_path=attribute_path, check_item='attribute')

if len(check_result_ps1+check_result_bd1 + check_result_ps2+check_result_bd2) == 0:
    data_merge(ps_mseg_c6_update_dir,
               bd_mseg_c6_update_dir,
               fuse_mseg_c6_update_dir)
    shutil.copy2(
        os.path.join(bd_mseg_c6_update_dir, 'class.txt'),
        os.path.join(fuse_mseg_c6_update_dir, 'class.txt'),
    )
    shutil.copy2(
        os.path.join(bd_mseg_c6_update_dir, 'attribute.yaml'),
        os.path.join(fuse_mseg_c6_update_dir, 'attribute.yaml'),
    )
else:
    print(f'find {check_result_ps1} and {check_result_bd1}')
    print(f'find {check_result_ps2} and {check_result_bd2}')

In [ ]:
dataset_dir = fuse_mseg_c6_dir
image_dir = os.path.join(dataset_dir, 'images')
labels_dir = os.path.join(dataset_dir, 'labels')
image_crop_dir = os.path.join(dataset_dir, 'images_crop')
class_file = os.path.join(dataset_dir, 'class.txt')
attribute_file = os.path.join(dataset_dir, 'attribute.yaml')
myolo_crop(image_dir, labels_dir, image_crop_dir, class_file,
           attribute_file=attribute_file, seg=True, annotation=True,
           save_method='attribute',
           crop_method='without_background_box_shape')

In [ ]:
dataset_dir = fuse_mseg_c6_update_dir
image_dir = os.path.join(dataset_dir, 'images')
labels_dir = os.path.join(dataset_dir, 'labels')
image_crop_dir = os.path.join(dataset_dir, 'images_crop')
class_file = os.path.join(dataset_dir, 'class.txt')
attribute_file = os.path.join(dataset_dir, 'attribute.yaml')
myolo_crop(image_dir, labels_dir, image_crop_dir, class_file,
           attribute_file=attribute_file, seg=True, annotation=True,
           save_method='attribute',
           crop_method='without_background_box_shape')

In [ ]:
dataset_dir = fuse_mseg_c6_dir
gt_dir=os.path.join(dataset_dir, "labels")
sta_dir=os.path.join(dataset_dir, "labels_sta")
class_path=os.path.join(dataset_dir, "class.txt")
attribute_path=os.path.join(dataset_dir, "attribute.yaml")
obj2img_path=os.path.join(dataset_dir, "images_crop.json")
sta_att_path=os.path.join(sta_dir, "sta_attribute.csv")
box_path=os.path.join(sta_dir, "sta_box.csv")
info_path=os.path.join(sta_dir, "info.csv")

In [ ]:
yolo_sta(
    img_dir=None,
    gt_dir=gt_dir,
    result_dir=sta_dir,
    class_path=class_path,
    attribute_path=attribute_path,
    seg=True,
)

In [ ]:
obj_record(sta_att_path, box_path, obj2img_path, info_path)

In [ ]:
dataset_dir = fuse_mseg_c6_update_dir

gt_dir=os.path.join(dataset_dir, "labels")
sta_dir=os.path.join(dataset_dir, "labels_sta")
class_path=os.path.join(dataset_dir, "class.txt")
attribute_path=os.path.join(dataset_dir, "attribute.yaml")
obj2img_path=os.path.join(dataset_dir, "images_crop.json")
sta_att_path=os.path.join(sta_dir, "sta_attribute.csv")
box_path=os.path.join(sta_dir, "sta_box.csv")
info_path=os.path.join(sta_dir, "info.csv")

In [ ]:
yolo_sta(
    img_dir=None,
    gt_dir=gt_dir,
    result_dir=sta_dir,
    class_path=class_path,
    attribute_path=attribute_path,
    seg=True,
)

In [ ]:
obj_record(sta_att_path, box_path, obj2img_path, info_path)

In [ ]:
dataset_dir = fuse_mseg_c6_dir
sta_dir=os.path.join(dataset_dir, "labels_sta")
info_path=os.path.join(sta_dir, "info.csv")
image_crop_dir = os.path.join(dataset_dir, 'images_crop')
val_path = os.path.join(dataset_dir, 'val.txt')

ref_split_class(ref_path=val_path,
                info_path=info_path,
                input_dir=image_crop_dir,
                defect_list = defect_list,
                level_list = level_list)


In [ ]:
dataset_dir = fuse_mseg_c6_update_dir
sta_dir=os.path.join(dataset_dir, "labels_sta")
info_path=os.path.join(sta_dir, "info.csv")
image_crop_dir = os.path.join(dataset_dir, 'images_crop')
val_path = os.path.join(dataset_dir, 'val.txt')
ref_split_class(ref_path=val_path,
                info_path=info_path,
                input_dir=image_crop_dir,
                defect_list = defect_list,
                level_list = level_list)

In [64]:
data_dir = r'/localnvme/data/billboard/fused_data/data1361_mseg_c6'
labels_dir = os.path.join(data_dir, 'labels')
images_dir = os.path.join(data_dir, 'images')
cls_val_dir = os.path.join(data_dir, 'cls_val207')
val_dir = os.path.join(r'/localnvme/project/ultralytics/runs/msegment', 'val207')
predict_dir = os.path.join(val_dir, 'labels')
labels_npy_dir = os.path.join(val_dir, 'labels_npy')
predict_npy_dir = os.path.join(val_dir, 'predicts_npy')
compare_dir = predict_dir+'val207_compare'

In [65]:
model_pred_compare(labels_dir, predict_dir, save_dir=compare_dir, seg=True, with_conf=False, with_att=True, threshold=0.5, att_num=4, image_width=1, image_height=1, labels_npy_dir=labels_npy_dir, predict_npy_dir=predict_npy_dir)

100%|██████████| 75/75 [00:03<00:00, 19.33it/s]

totally 546 box are counted


In [66]:
select_by_predict(compare_dir, images_dir, cls_val_dir,
                  defect_list = defect_list,
                  level_list = level_list)

100%|██████████| 75/75 [00:13<00:00,  5.41it/s]


In [67]:
data_dir = r'/localnvme/data/billboard/fused_data/data1361_mseg_c6_check0618'
labels_dir = os.path.join(data_dir, 'labels')
images_dir = os.path.join(data_dir, 'images')
cls_val_dir = os.path.join(data_dir, 'cls_val208')
val_dir = os.path.join(r'/localnvme/project/ultralytics/runs/msegment', 'val208')
predict_dir = os.path.join(val_dir, 'labels')
labels_npy_dir = os.path.join(val_dir, 'labels_npy')
predict_npy_dir = os.path.join(val_dir, 'predicts_npy')
compare_dir = predict_dir+'val208_compare'

In [68]:
model_pred_compare(labels_dir, predict_dir, save_dir=compare_dir, seg=True, with_conf=False, with_att=True, threshold=0.5, att_num=4, image_width=1, image_height=1, labels_npy_dir=labels_npy_dir, predict_npy_dir=predict_npy_dir)

100%|██████████| 75/75 [00:03<00:00, 19.57it/s]

totally 545 box are counted


In [69]:
select_by_predict(compare_dir, images_dir, cls_val_dir,
                  defect_list = defect_list,
                  level_list = level_list)

100%|██████████| 75/75 [00:14<00:00,  5.11it/s]
